In [ ]:
BiocManager::install('mixOmics')
BiocManager::install("pcaMethods")
install.packages("devtools")
devtools::install_github("alexpghayes/distributions3")
install.packages('fitdistrplus')
install.packages("useful")

library(distributions3)
library(fitdistrplus)
library(mixOmics)
library(data.table)
library(pcaMethods)
library(useful)
library(ggrepel)

In [ ]:
cca_run<-function(gene, drug, name){

    cca = rcc(gene, drug, ncomp = 20, method = 'shrinkage')
    
    joint_variate = (cca$variates$X+cca$variates$Y)/2
    
    projx<-data.frame(var=colnames(cca$X), 
                      cor(cca$X, cca$variates$X+cca$variates$Y, use="pairwise"))
    projy<-data.frame(var=colnames(cca$Y), 
                      cor(cca$Y, cca$variates$X+cca$variates$Y, use="pairwise"))
    
    fwrite(data.frame(cca$cor), paste0(name,"_cca.csv"), sep ="\t", col.names = T, row.names = F, quote= F)
    fwrite(joint_variate, paste0(name,"_variate.csv"), sep ="\t", col.names = T, row.names = F, quote= F)
    fwrite(projx,paste0(name,"_gene_loading.csv"), sep ="\t", col.names = T, row.names = F, quote= F)
    fwrite(projy,paste0(name,"_drug_loading.csv"), sep ="\t", col.names = T, row.names = F, quote= F)

}
pca_run<-function(gene_data, gene_name,
                  name, 
                  nPC_comp = 20, top_comp = 10){

    c_pca <- pca(gene_data, 
                 method="ppca", 
                 nPcs=nPC_comp, seed=123)
    
    c_pcascore_loading<-data.frame(gene=gene_name,
                                   c_pca@loadings)
    
    c_pcascore<-data.frame(c_pca@scores)
    
    fwrite(c_pcascore_loading, paste0(name,"_pca_loading.csv"), sep="\t", row.names = F, col.names = T, quote=F) 
    fwrite(c_pcascore, paste0(name,"_pca_score.csv"), sep="\t", row.names = F, col.names = T, quote=F) 
    

}

drug_potentiality<-function(name,top_comp = 10){
    
    pca_gene_loading = fread(paste0(name,"_pca_loading.csv") )
    cca_gene_loading = fread(paste0(name,"_gene_loading.csv") )
    
    gene_name = cca_gene_loading$var
    
    ###Max PCA score 
#     c_pca <- pca(gene_data, method="ppca", nPcs=nPC_comp, seed=123)
#     c_pcascore_loading<-data.frame(gene=gene_name,
#                                    c_pca@loadings)
#     cpca_max= data.frame(gene = gene_name , 
#                          value = apply(abs(data.frame(c_pcascore_loading)[, 2:(top_comp+1)]), 1, max))
                                     
    ###Max PCA score
    c_pcamax= data.frame(gene = gene_name , 
                         value = apply(abs(data.frame(pca_gene_loading)[, 2:(top_comp+1)]), 1, max))
    
    ###Max CCA score 
    c_ccamax= data.frame(gene = gene_name , 
                         value = apply(abs(data.frame(cca_gene_loading)[, 2:(top_comp+1)]), 1, max))

    
    cca_file = c_ccamax
    cca_file$abs<-abs(cca_file$value)
    fit_ln <- fitdist(cca_file$abs , "lnorm")
    c_cca<-plnorm(cca_file$abs, meanlog=fit_ln$estimate[1], sdlog=fit_ln$estimate[2])

    pca_file = c_pcamax
    pca_file$abs<-abs(pca_file$value)
    fit_ln <- fitdist(pca_file$abs , "lnorm")
    c_pca<-plnorm(pca_file$abs, meanlog=fit_ln$estimate[1], sdlog=fit_ln$estimate[2])

    polar<-cart2pol(cca_file$abs, pca_file$abs, degrees = T)

    end<-merge(cca_file, pca_file, by="gene")
    colnames(end)<-c("gene","cca_og", "cca_abs","pca_og","pca_abs")
    end<-data.frame(end, polar,
                    cca_quant=c_cca, 
                    pca_quant=c_pca, 
                    dif=c_cca - c_pca)
                                     
    fwrite(end, paste0(name,"_potential_score.csv"), sep="\t", row.names = F, col.names = T, quote=F) 

}

In [ ]:
plot_loading<-function(cca_gene_loading, comp1, comp2, name){
  
  g<-ggplot(cca_gene_loading, aes_string(x = comp1, y = comp2, alpha = 0.5 ,label = "plot_var")) +
    geom_text(alpha = 1)+
    geom_point( size=I(4),show.legend = F)  +
    theme(axis.line = element_line(colour = "black"),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.border = element_blank(),
          panel.background = element_blank(),
          axis.title.x = element_text(size=20),
          axis.title.y = element_text(size=20),
          text = element_text(size=15 , family="Arial", )
    )+geom_vline(xintercept = c(0.3, -0.3) , linetype = "dashed")+ geom_hline(yintercept = c(0.3,-0.3), linetype = "dashed")+
    labs(x = comp1, y = comp2, title=name)
  ggsave(paste0(name,comp1,comp2,"_loading.png"), g, width = 10, height = 10)
  
}
plot_drug_loading<-function(cca_gene_loading, comp1, comp2, name){
  
  g<-ggplot(cca_gene_loading, aes_string(x = comp1, y = comp2, alpha = 0.5 ,label = "plot_var")) +
    geom_text(alpha = 1)+
    geom_point( size=I(4),show.legend = F)  +
    theme(axis.line = element_line(colour = "black"),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.border = element_blank(),
          panel.background = element_blank(),
          axis.title.x = element_text(size=20),
          axis.title.y = element_text(size=20),
          text = element_text(size=15 , family="Arial", )
    )+geom_vline(xintercept = c(0.3, -0.3) , linetype = "dashed")+ geom_hline(yintercept = c(0.3,-0.3), linetype = "dashed")+
    labs(x = comp1, y = comp2, title=name)
  ggsave(paste0(name,comp1,comp2,"_drug_loading.png"), g, width = 10, height = 10)
  
}

plot_pca_loading<-function(cca_gene_loading, comp1, comp2, name){
  
  g<-ggplot(cca_gene_loading, aes_string(x = comp1, y = comp2, alpha = 0.5 ,label = "plot_var")) +
    geom_text(alpha = 1)+
    geom_point( size=I(4),show.legend = F)  +
    theme(axis.line = element_line(colour = "black"),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.border = element_blank(),
          panel.background = element_blank(),
          axis.title.x = element_text(size=20),
          axis.title.y = element_text(size=20),
          text = element_text(size=15 , family="Arial", )
    ) + 
    labs(x = comp1, y = comp2, title=name)
  ggsave(paste0(name,comp1,comp2,"_pca_loading.png"), g, width = 10, height = 10)
  
}
plot_variate<-function(cca_variate, comp1, comp2, name){
  
  g<-ggplot(data = cca_variate , aes_string(x= comp1, y = comp2))+
    geom_point(size = I(6),aes(color = plot_type, alpha = alpha), show.legend = T)+
    scale_color_manual("Cancer types",
                       values=c("Leukemia"="purple", 
                                "Myeloma"="hotpink",
                                "Lymphoma"="mediumvioletred",
                                'Small Cell Lung Cancer' = "forestgreen",
                                "Non Small Cell Lung Cancer"= "yellowgreen", 
                                "Skin Cancer" = "darkorange", 
                                "Head and Neck Cancer" = "brown",
                                "Brain Cancer" = "skyblue",
                                "Neuroblastoma" = "navyblue",
                                "Gastric Cancer" = "tan2",
                                "Esophageal Cancer" = "darkgreen",
                                "Bone Cancer" = "cyan",
                                "Breast Cancer" = "gold" ,
                                'Gastric Cancer' = "darkblue",
                                "Bile Duct Cancer" = "green",
                                "Ovarian Cancer" = "pink",
                                "Lung Cancer" = "yellow",
                                "X_Others" = "grey"
                       ))+
    theme( axis.line = element_line(colour = "black"),
           panel.grid.major = element_blank(),
           panel.grid.minor = element_blank(),
           panel.border = element_blank(),
           panel.background = element_blank(),
           text = element_text(size=20),
           axis.title.x = element_text(size=20,margin = margin(t = 5, r = 0 , b = 0, l = 0)),
           axis.title.y = element_text(size=20,margin = margin(t = 0, r = 18 , b = 0, l = 0)),
           axis.text.x = element_text(size=15),
           axis.text.y = element_text(size=15))+
    labs(y=comp2, x=comp1, title = name)
  
  ggsave(paste0(name,comp1,comp2,"_variate.png"), g, width = 17,height = 10) 
}

prelim_analysis<-function(name, info, comp1 =1 , comp2= 2){

pca_gene_loading = fread(paste0(name,"_pca_loading.csv") )
cca_gene_loading = fread(paste0(name,"_gene_loading.csv") )
cca_drug_loading = fread(paste0(name,"_drug_loading.csv") )
cca_variate = fread(paste0(name,"_variate.csv") )
cca_variate = cbind(info, cca_variate)
cca_variate$primary_disease = cca_variate$primary_disease
cca_variate$subtype = cca_variate$Subtype
cca_variate[which(cca_variate$subtype == "Small Cell Lung Cancer (SCLC)"),'primary_disease'] = "Small Cell Lung Cancer"
cca_variate$plot_type = cca_variate$primary_disease
cca_variate$plot_type = ifelse(cca_variate$plot_type == "Leukemia" , "Leukemia",
                               ifelse(cca_variate$plot_type == "Small Cell Lung Cancer", 'Small Cell Lung Cancer', 
                                      ifelse(cca_variate$plot_type == "Myeloma" , "Myeloma", 
                                             ifelse(cca_variate$plot_type == "Lymphoma", "Lymphoma", 
                                                    ifelse(cca_variate$plot_type == "Lung Cancer", "Lung Cancer", 
                                                           ifelse(cca_variate$plot_type == "Head and Neck Cancer", "Head and Neck Cancer", 
                                                                  ifelse(cca_variate$plot_type == "Skin Cancer", "Skin Cancer",
                                                                         ifelse(cca_variate$plot_type == "Brain Cancer", 'Brain Cancer',
                                                                                ifelse(cca_variate$plot_type == "Neuroblastoma", 'Neuroblastoma',
                                                                                       ifelse(cca_variate$plot_type == "Gastric Cancer", 'Gastric Cancer',
                                                                                              ifelse(cca_variate$plot_type == "Bile Duct Cancer", 'Bile Duct Cancer',
                                                                                                     ifelse(cca_variate$plot_type == "Breast Cancer", 'Breast Cancer',
                                                                                                            ifelse(cca_variate$plot_type == "Ovarian Cancer", 'Ovarian Cancer',
                                                                                                                   "X_Others" )))))))))))))
cca_variate$alpha = ifelse(cca_variate$plot_type == "X_Others", 0.9, 1)

top_gene_neg = cca_gene_loading[order(-cca_gene_loading$X1)]$var[1:20]
top_gene_neg = cca_gene_loading[order(cca_gene_loading$X1)]$var[1:20]

top_gene_pos2 = cca_gene_loading[order(-cca_gene_loading$X2)]$var[1:20]
top_gene_neg2 = cca_gene_loading[order(cca_gene_loading$X2)]$var[1:20]

cca_gene_loading$plot_var =cca_gene_loading$var 
cca_gene_loading[!cca_gene_loading$var%in%(c(top_gene_pos, top_gene_neg, 
                                             top_gene_pos2, top_gene_neg2))]$plot_var = ""

top_gene_pos = cca_drug_loading[order(-cca_drug_loading$X1)]$var[1:20]
top_gene_neg = cca_drug_loading[order(cca_drug_loading$X1)]$var[1:20]

top_gene_pos2 = cca_drug_loading[order(-cca_drug_loading$X2)]$var[1:20]
top_gene_neg2 = cca_drug_loading[order(cca_drug_loading$X2)]$var[1:20]

cca_drug_loading$plot_var =cca_drug_loading$var 
cca_drug_loading[!cca_drug_loading$var%in%(c(top_gene_pos, top_gene_neg, 
                                             top_gene_pos2, top_gene_neg2))]$plot_var = ""


top_gene_pos = pca_gene_loading[order(-pca_gene_loading$PC1)]$gene[1:20]
top_gene_neg = pca_gene_loading[order(pca_gene_loading$PC1)]$gene[1:20]

top_gene_pos2 = pca_gene_loading[order(-pca_gene_loading$PC2)]$gene[1:20]
top_gene_neg2 = pca_gene_loading[order(pca_gene_loading$PC2)]$gene[1:20]

pca_gene_loading$plot_var =pca_gene_loading$gene 
pca_gene_loading[!pca_gene_loading$gene%in%(c(top_gene_pos, top_gene_neg, 
                                              top_gene_pos2, top_gene_neg2))]$plot_var = ""

plot_variate(cca_variate  = cca_variate, comp1 = paste0("V",comp1), comp2 = paste0("V",comp2), name = name)
plot_loading(cca_gene_loading = cca_gene_loading, comp1 = paste0("X",comp1), comp2 = paste0("X",comp2), name = name)
plot_drug_loading(cca_gene_loading = cca_drug_loading, comp1 = paste0("X",comp1), comp2 = paste0("X",comp2), name = name)
#plot_pca_loading(cca_gene_loading = pca_gene_loading, comp = paste0("PC",comp1),comp2 = paste0("PC",comp2), name = name )

}

### Example 

In [ ]:
info = fread("crispr_achilles_info_ctrp.csv")
drug = fread("ctrp_crispr_achilles_impute.csv")
gene = fread("achilles_crispr_ctrp_impute.csv")
gene_nonimpute = fread("achilles_crispr_ctrp.csv")

cca_run(gene, drug, name = "22q1_rnai_crispr_ctrp")
pca_run(gene_data = gene_nonimpute ,
        gene_name = colnames(gene_nonimpute),
        name = "22q1_rnai_crispr_ctrp")
drug_potentiality(name = '22q1_rnai_crispr_ctrp',
                  top_comp = 10)